# Interactive notebook of `helmpy` examples

The mathematics of the model which `helmpy` uses are described in detail here: [https://doi.org/10.1101/2019.12.17.19013490]

In this notebook we will run through all of the features in `helmpy` as well as providing insights into its construction along the way. Throughout we shall be running the algorithm only once to generate our plots, however for best results it is recommended to rerun `helmpy` a few more times (fewer than $10$ iterations) and to combine the output. Because a number of realisations may be run at once (e.g., $250$), this can lead to very good statistics in the output (overall $10 \times 250$ realisations) with high efficiency.

First we must tell the system where the `helmpy` directory is...

In [ ]:
import sys
path_to_helmpy = '/Users/Rob/work/helmpy' # Give your path to helmpy here
sys.path.append(path_to_helmpy + '/source/') 
from helmpy import helmpy

# These modules are not necessary to run helmpy alone but will be useful for our demonstrations

# LEAVE THESE IMPORTS COMMENTED AS THEY ARE FOR PRODUCING LaTeX-STYLE FIGURES ONLY
#import matplotlib as mpl
#mpl.use('Agg')
#mpl.rc('font',family='CMU Serif')
#mpl.rcParams['xtick.labelsize'] = 15
#mpl.rcParams['ytick.labelsize'] = 15
#mpl.rcParams['axes.labelsize'] = 20
#from matplotlib import rc
#rc('text',usetex=True)
#rc('text.latex',preamble=r'\usepackage{mathrsfs}')
#rc('text.latex',preamble=r'\usepackage{sansmath}')
# LEAVE THESE IMPORTS COMMENTED AS THEY ARE FOR PRODUCING LaTeX-STYLE FIGURES ONLY

import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import time

# 1. Soil-transmitted helminths (STH)

## 1.1 Running basic simulations in three steps

### Step 1

Initialise the `helmpy` class by specifying a disease type and working directory...

In [ ]:
hp = helmpy(
            'STH',                              # Set the disease type - types available are: 'STH', 'SCH' and 'LF'
            path_to_helmpy,                     # Set the path to the working directory
            suppress_terminal_output=False      # Set this to 'True' to remove terminal messages
            )  

### Step 2

Set the parameters and initial conditions for the simulation. List sizes are variable depending on the number of groupings one wishes to apply. Hence age, gender and all other stratifications within a cluster may be programmed in straightforwardly and each grouping may be identified within a given cluster by a spatial index. The available basic options to vary are...

In [ ]:
hp.parameter_dictionary['mu'] = [0.014,0.014]   # Human death rate (per year)
hp.parameter_dictionary['mu1'] = [0.5,0.5]      # Adult worm death rate (per year)
hp.parameter_dictionary['mu2'] = [26.0,26.0]    # Reservoir (eggs and larvae) death rate (per year)
hp.parameter_dictionary['R0'] = [3.5,2.1]       # Basic reproduction number within grouping
#hp.parameter_dictionary['R0'] = [3.5,3.0]       # Basic reproduction number within grouping
hp.parameter_dictionary['k'] = [0.3,0.5]        # Inverse-clumping factor within grouping
hp.parameter_dictionary['gam'] = [0.08,0.08]    # Density dependent fecundity: z = exp(-gam)
hp.parameter_dictionary['Np'] = [300,350]       # Number of people within grouping   
hp.parameter_dictionary['spi'] = [1,2]          # Spatial index number of grouping

hp.initial_conditions['M'] = [2.9,2.1]          # Initial mean total worm burden within grouping
#hp.initial_conditions['M'] = [2.9,3.1]          # Initial mean total worm burden within grouping
hp.initial_conditions['FOI'] = [1.25,1.1]       # Initial force of infection (per year) within grouping
#hp.initial_conditions['FOI'] = [1.25,1.5]       # Initial force of infection (per year) within grouping

So we are running two non-interacting clusters each with a single grouping of people which both contain different numbers of people, different initial conditions, different parasite clumping (controlled through $k$) and different overall values of parasite uptake: here combined into a parameter we call `R0` - note that these are different to the true $R_0$ value in the cluster which can be computed from the weighted sum of these sub-`R0` values.

### Step 3

Run the simulation by setting the key numerical parameters. For comparison we will also run the mean field and mean field stochastic codes (which are much faster) alongside the full simulation...

In [ ]:
runtime = 100.0                               # Set the total time of the run in years
realisations = 250                            # Set the number of stochastic realisations for the model
do_nothing_timescale = 0.01                   # Set a timescale (in years) short enough such that an individual 
                                              # is expected to stay in the same state
    
output_filename = 'default_example'           # Set a filename for the data to be output in data/
            
output_timesteps = [1000,2000,3000,9000]      # Optional - output binned worm burdens over whole population 
                                              # and realisations after a specified number of steps in time  
hp.run_full_stoch(         
                  runtime,          
                  realisations,  
                  do_nothing_timescale,
                  output_filename,  
                  timesteps_snapshot=output_timesteps
                  )

hp.run_meanfield_stoch(         
                       runtime, 
                       realisations,
                       do_nothing_timescale,                
                       'meanfield_stoch_' + output_filename,
                       timesteps_snapshot=output_timesteps
                       )

hp.run_meanfield(         
                 runtime,  
                 do_nothing_timescale,                
                 'meanfield_' + output_filename       
                 )

Having run all three models, let us now read in the data and compare the ensemble mean and variance of the mean worm burden realisations in each cluster.

First we compare the predictions of the mean field model to those of the full simulation...

In [ ]:
example_output_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + '.txt')
example_meanfield_output_data = np.loadtxt(path_to_helmpy + '/data/' + 'meanfield_' + output_filename + '.txt')

# Mean of ensemble in cluster 1
plt.plot(example_output_data.T[0],example_output_data.T[1],color='r',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][0]) + r'$')

# Mean of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[2],color='b',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][1]) + r'$')

# Mean field of ensemble in cluster 1
plt.plot(example_meanfield_output_data.T[0],example_meanfield_output_data.T[1],color='r',alpha=0.4) 

# Mean field of ensemble in cluster 2
plt.plot(example_meanfield_output_data.T[0],example_meanfield_output_data.T[2],color='b',alpha=0.4) 

axes = plt.gca()
axes.set_ylim([2.5,7.5])
axes.set_ylabel(r'$M(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12)
plt.savefig(path_to_helmpy + '/plots/Mean_meanfield_comp.png',format='png',dpi=500)
plt.show()

In [ ]:
example_output_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + '.txt')
example_meanfield_output_data = np.loadtxt(path_to_helmpy + '/data/' + 'meanfield_' + output_filename + '.txt')

# Variance of ensemble in cluster 1
plt.plot(example_output_data.T[0],example_output_data.T[3],color='r',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][0]) + r'$')

# Variance of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[4],color='b',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][1]) + r'$')


# Variance from mean field of ensemble in cluster 1
plt.plot(example_meanfield_output_data.T[0],example_meanfield_output_data.T[3],color='r',alpha=0.4)

# Variance from mean field of ensemble in cluster 2
plt.plot(example_meanfield_output_data.T[0],example_meanfield_output_data.T[4],color='b',alpha=0.4)

axes = plt.gca()
axes.set_ylim([0.0,2.5])
axes.set_ylabel(r'$V(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12)
plt.savefig(path_to_helmpy + '/plots/Var_meanfield_comp.png',format='png',dpi=500)
plt.show()

Secondly, we compare the predictions of a fast stochastic model built from the mean field code to those of the full simulation...

In [ ]:
example_output_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + '.txt')
example_meanfield_stoch_data = np.loadtxt(path_to_helmpy + '/data/' + 'meanfield_stoch_' + output_filename + '.txt')

# Mean of ensemble in cluster 1
plt.plot(example_output_data.T[0],example_output_data.T[1],color='r',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][0]) + r'$')

# Mean of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[2],color='b',\
         label=r'$R_0 =' + str(hp.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hp.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hp.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hp.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hp.initial_conditions['FOI'][1]) + r'$')

# 68% CLs of ensemble in cluster 1
plt.plot(example_output_data.T[0],example_output_data.T[5],color='r')
plt.plot(example_output_data.T[0],example_output_data.T[7],color='r')

# 68% CLs of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[6],color='b')
plt.plot(example_output_data.T[0],example_output_data.T[8],color='b')

# Mean of mean-field stochastic ensemble in cluster 1
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[1],color='r',alpha=0.4)

# Mean of mean-field stochastic ensemble in cluster 2
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[2],color='b',alpha=0.4)

# 68% CLs of mean-field stochastic ensemble in cluster 1
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[5],color='r',alpha=0.4)
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[7],color='r',alpha=0.4)

# 68% CLs of mean-field stochastic ensemble in cluster 2
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[6],color='b',alpha=0.4)
plt.plot(example_meanfield_stoch_data.T[0],example_meanfield_stoch_data.T[8],color='b',alpha=0.4)

axes = plt.gca()
axes.set_ylim([0.0,9.5])
axes.set_ylabel(r'$m(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12, loc = 9)
plt.savefig(path_to_helmpy + '/plots/meanfield_stoch_comp.png',format='png',dpi=500)
plt.show()

Now let us also have a look at some snapshots in time of the mean worm burden distribution from the full simulation...

In [ ]:
time_values = [1000,2000,3000,9000]
alphlist = [0.2,0.4,0.6,1.0]

for ti in range(0,len(time_values)):
    wb_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + \
                         '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_1.txt')
    bo = np.histogram((np.sum(wb_data,axis=1)/len(wb_data.T)).astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='r',alpha=alphlist[ti])
    
for ti in range(0,len(time_values)):
    wb_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + \
                         '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_2.txt')
    bo = np.histogram((np.sum(wb_data,axis=1)/len(wb_data.T)).astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='b',alpha=alphlist[ti])

axes = plt.gca()
axes.set_xlim([0.0,8.0])
plt.show()

...and the equivalent from the mean field stochastic code...

In [ ]:
time_values = [1000,2000,3000,9000]
alphlist = [0.2,0.4,0.6,1.0]

for ti in range(0,len(time_values)):
    mwb_data = np.loadtxt(path_to_helmpy + '/data/'  + 'meanfield_stoch_' + output_filename + \
                          '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_1.txt')
    bo = np.histogram(mwb_data.astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='r',alpha=alphlist[ti])
    
for ti in range(0,len(time_values)):
    mwb_data = np.loadtxt(path_to_helmpy + '/data/'  + 'meanfield_stoch_' + output_filename + \
                          '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_2.txt')
    bo = np.histogram(mwb_data.astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='b',alpha=alphlist[ti])

axes = plt.gca()
axes.set_xlim([0.0,8.0])
plt.show()

## 1.2 Adding inter-cluster migration

To begin with, let us create a new instance of `helmpy` with the same parameters and initial conditions set for comparison in order to continue working distinctly from the previous section...

In [ ]:
hpmig = helmpy(
              'STH',                              # Set the disease type - types available are: 'STH', 'SCH' and 'LF'
              path_to_helmpy,                     # Set the path to the working directory
              suppress_terminal_output=False      # Set this to 'True' to remove terminal messages
              ) 

hpmig.parameter_dictionary['mu'] = [0.014,0.014]   # Human death rate (per year)
hpmig.parameter_dictionary['mu1'] = [0.5,0.5]      # Adult worm death rate (per year)
hpmig.parameter_dictionary['mu2'] = [26.0,26.0]    # Reservoir (eggs and larvae) death rate (per year)
hpmig.parameter_dictionary['R0'] = [3.5,2.1]       # Basic reproduction number within grouping
hpmig.parameter_dictionary['k'] = [0.3,0.5]        # Inverse-clumping factor within grouping
hpmig.parameter_dictionary['gam'] = [0.08,0.08]    # Density dependent fecundity: z = exp(-gam)
hpmig.parameter_dictionary['Np'] = [300,350]       # Number of people within grouping   
hpmig.parameter_dictionary['spi'] = [1,2]          # Spatial index number of grouping

hpmig.initial_conditions['M'] = [2.9,2.1]          # Initial mean total worm burden within grouping
hpmig.initial_conditions['FOI'] = [1.25,1.1]       # Initial force of infection (per year) within grouping

Having run the basic simulations above for two separate clusters, let us now introduce an inter-cluster migration effect by specifying non-zero migratory rate matrices with the following definitions: Components $(r_+)_{ij}$ and $(r_-)_{ij}$ are the migration rates (per year) of individuals in to cluster $i$ from cluster $j$ and out of cluster $i$ from cluster $j$, respectively. Hence, the matrices for two clusters are

$${\bf r}_+ = \begin{bmatrix} (r_+)_{11} & (r_+)_{12} \\ (r_+)_{21} & (r_+)_{22}\end{bmatrix}\,,\qquad {\bf r}_- = \begin{bmatrix} (r_-)_{11} & (r_-)_{12} \\ (r_-)_{21} & (r_-)_{22}\end{bmatrix}\,,$$

and one may deduce that $(r_+)_{ij}=(r_-)_{ji}$. Let us now set these matrices in the parameters of our `helmpy` instance...

In [ ]:
hpmig.parameter_dictionary['r+'] = [[0.0,0.0],[52.0,0.0]]  # Migration matrix - the migration rate in (per year)
hpmig.parameter_dictionary['r-'] = [[0.0,52.0],[0.0,0.0]]  # Migration matrix - the migration rate out (per year)
hpmig.parameter_dictionary['Nm'] = [1]                     # Number of migrants per event (global parameter)

Now that the migrations have been set, we may simply rerun the code as before...

In [ ]:
runtime = 100.0                               # Set the total time of the run in years
realisations = 250                            # Set the number of stochastic realisations for the model
do_nothing_timescale = 0.01                   # Set a timescale (in years) short enough such that an individual 
                                              # is expected to stay in the same state
    
output_filename = 'default_example'           # Set a filename for the data to be output in data/
            
output_timesteps = [1000,2000,3000,9000]      # Optional - output binned worm burdens over whole population 
                                              # and realisations after a specified number of steps in time  

hpmig.run_full_stoch(         
                    runtime,          
                    realisations,  
                    do_nothing_timescale,
                    'mig_' + output_filename,  
                    timesteps_snapshot=output_timesteps
                    )

In [ ]:
example_output_data = np.loadtxt(path_to_helmpy + '/data/' + output_filename + '.txt')
example_output_data_mig = np.loadtxt(path_to_helmpy + '/data/' + 'mig_' + output_filename + '.txt')

# Mean of ensemble in cluster 1 with migration
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[1],color='r',\
         label=r'$R_0 =' + str(hpmig.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hpmig.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hpmig.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hpmig.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hpmig.initial_conditions['FOI'][0]) + r'$')

# Mean of ensemble in cluster 2 with migration
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[2],color='b',\
         label=r'$R_0 =' + str(hpmig.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hpmig.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hpmig.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hpmig.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hpmig.initial_conditions['FOI'][1]) + r'$')

# 68% CLs of ensemble in cluster 1 with migration
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[5],color='r')
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[7],color='r')

# 68% CLs of ensemble in cluster 2 with migration
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[6],color='b')
plt.plot(example_output_data_mig.T[0],example_output_data_mig.T[8],color='b')

# Mean of ensemble in cluster 1 
plt.plot(example_output_data.T[0],example_output_data.T[1],color='r',alpha=0.4)

# Mean of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[2],color='b',alpha=0.4)

# 68% CLs of ensemble in cluster 1
plt.plot(example_output_data.T[0],example_output_data.T[5],color='r',alpha=0.4)
plt.plot(example_output_data.T[0],example_output_data.T[7],color='r',alpha=0.4)

# 68% CLs of ensemble in cluster 2
plt.plot(example_output_data.T[0],example_output_data.T[6],color='b',alpha=0.4)
plt.plot(example_output_data.T[0],example_output_data.T[8],color='b',alpha=0.4)

axes = plt.gca()
axes.set_ylim([0.0,8.5])
axes.set_ylabel(r'$m(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12, loc = 9)
plt.text(50.0,0.5,r'$(r_+,r_-)=(' + \
         str(np.round(hpmig.parameter_dictionary['r+'][0][1]/ \
                      hpmig.parameter_dictionary['mu2'][0],decimals=1)) + '\mu_2,' + \
         str(np.round(hpmig.parameter_dictionary['r-'][0][1]/ \
                      hpmig.parameter_dictionary['mu2'][0],decimals=1)) + '\mu_2)$', \
          color='r',fontsize=15)
plt.savefig(path_to_helmpy + '/plots/mig_comp.png',format='png',dpi=500)
plt.show()

Now let once again have a look at some snapshots in time of the mean worm burden distribution from the full simulation...

In [ ]:
time_values = [1000,2000,3000,9000]
alphlist = [0.2,0.4,0.6,1.0]

for ti in range(0,len(time_values)):
    wb_data = np.loadtxt(path_to_helmpy + '/data/' + 'mig_' + output_filename + \
                         '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_1.txt')
    bo = np.histogram((np.sum(wb_data,axis=1)/len(wb_data.T)).astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='r',alpha=alphlist[ti])
    
for ti in range(0,len(time_values)):
    wb_data = np.loadtxt(path_to_helmpy + '/data/' + 'mig_' + output_filename + \
                         '_snapshot_timestep_' + str(time_values[ti]) + '_cluster_2.txt')
    bo = np.histogram((np.sum(wb_data,axis=1)/len(wb_data.T)).astype(float),bins='fd',density=True)
    box = 0.5*(bo[1][:len(bo[0])]+bo[1][1:len(bo[0])+1])
    boy = bo[0]
    plt.plot(box[boy!=0.0],(boy/max(boy))[boy!=0.0],color='b',alpha=alphlist[ti])

axes = plt.gca()
axes.set_xlim([0.0,8.0])
plt.show()

## 1.3 Adding treatment rounds

The `helmpy` class models rounds of mass drug administration (MDA) through Bernoulli trials with a given probability of reducing an individual's worm burden to 0. This is effectively a 'random compliance' model. Let us use this feature to examine the influence of migration on the efficacy of treatment using the system setup from before...

In [ ]:
# Change the Boolean to see the effect of migration for simplicity...
with_migration = False

hptrt = helmpy(
              'STH',                              # Set the disease type - types available are: 'STH', 'SCH' and 'LF'
              path_to_helmpy,                     # Set the path to the working directory
              suppress_terminal_output=False      # Set this to 'True' to remove terminal messages
              ) 

hptrt.parameter_dictionary['mu'] = [0.014,0.014]   # Human death rate (per year)
hptrt.parameter_dictionary['mu1'] = [0.5,0.5]      # Adult worm death rate (per year)
hptrt.parameter_dictionary['mu2'] = [26.0,26.0]    # Reservoir (eggs and larvae) death rate (per year)
hptrt.parameter_dictionary['R0'] = [3.5,2.1]       # Basic reproduction number within grouping
hptrt.parameter_dictionary['k'] = [0.3,0.5]        # Inverse-clumping factor within grouping
hptrt.parameter_dictionary['gam'] = [0.08,0.08]    # Density dependent fecundity: z = exp(-gam)
hptrt.parameter_dictionary['Np'] = [300,350]       # Number of people within grouping   
hptrt.parameter_dictionary['spi'] = [1,2]          # Spatial index number of grouping

hptrt.initial_conditions['M'] = [2.9,2.1]          # Initial mean total worm burden within grouping
hptrt.initial_conditions['FOI'] = [1.25,1.1]       # Initial force of infection (per year) within grouping

if with_migration == True:
    hptrt.parameter_dictionary['r+'] = [[0.0,0.0],[52.0,0.0]] # Migration matrix - the migration rate in (per year)
    hptrt.parameter_dictionary['r-'] = [[0.0,52.0],[0.0,0.0]] # Migration matrix - the migration rate out (per year)
    hptrt.parameter_dictionary['Nm'] = [10]                   # Number of migrants per event (global parameter) 

Having initialised a new instance, let us define 3 rounds of MDA with 60% coverage at years 15, 16 and 17 to see the effect this has on the disease...

In [ ]:
treatment_coverages = [[0.6,0.6,0.6],         # A list of lists matching the chosen groupings which gives 
                       [0.6,0.6,0.6]]         # the effective coverage fraction in each

treatment_times = [15.0,16.0,17.0]            # A list of treatment times for all clusters

hptrt.add_treatment_prog(
                        treatment_times,         
                        treatment_coverages=treatment_coverages,
                        drug_efficacy=1.0
                        )

Now we can run the simulation which will output both the mean worm burden dynamics (as before) and the final prevalence realisations at the last round of treatment and at the absolute end of the runs...

In [ ]:
runtime = 100.0                               # Set the total time of the run in years
realisations = 250                            # Set the number of stochastic realisations for the model
do_nothing_timescale = 0.01                   # Set a timescale (in years) short enough such that an individual 
                                              # is expected to stay in the same state
    
output_filename = 'default_example'           # Set a filename for the data to be output in data/
            
output_timesteps = [1000,2000,3000,9000]      # Optional - output binned worm burdens over whole population 
                                              # and realisations after a specified number of steps in time

if with_migration == False:   
    hptrt.run_full_stoch(         
                        runtime,          
                        realisations,  
                        do_nothing_timescale,
                        'trt_' + output_filename,  
                        timesteps_snapshot=output_timesteps
                        )
    
if with_migration == True:   
    hptrt.run_full_stoch(         
                        runtime,          
                        realisations,  
                        do_nothing_timescale,
                        'mig_trt_' + output_filename,  
                        timesteps_snapshot=output_timesteps
                        )

In [ ]:
example_output_data_mig_trt = np.loadtxt(path_to_helmpy + '/data/' + 'mig_trt_' + output_filename + '.txt')
example_output_data_trt = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + '.txt')

# Mean of ensemble in cluster 1 with migration
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[1],color='r',\
         label=r'$R_0 =' + str(hptrt.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hptrt.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hptrt.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hptrt.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hptrt.initial_conditions['FOI'][0]) + r'$')

# Mean of ensemble in cluster 2 with migration
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[2],color='b',\
         label=r'$R_0 =' + str(hptrt.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hptrt.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hptrt.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hptrt.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hptrt.initial_conditions['FOI'][1]) + r'$')

# 68% CLs of ensemble in cluster 1 with migration
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[5],color='r')
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[7],color='r')

# 68% CLs of ensemble in cluster 2 with migration
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[6],color='b')
plt.plot(example_output_data_mig_trt.T[0],example_output_data_mig_trt.T[8],color='b')

# Mean of ensemble in cluster 1 
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[1],color='r',alpha=0.4)

# Mean of ensemble in cluster 2
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[2],color='b',alpha=0.4)

# 68% CLs of ensemble in cluster 1
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[5],color='r',alpha=0.4)
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[7],color='r',alpha=0.4)

# 68% CLs of ensemble in cluster 2
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[6],color='b',alpha=0.4)
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[8],color='b',alpha=0.4)

axes = plt.gca()
axes.set_ylim([0.0,8.5])
axes.set_ylabel(r'$m(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12, loc = 9)
#plt.text(50.0,1.0,r'$(r_+,r_-)=(' + str(hptrt.parameter_dictionary['r+'][0][1]) + ',' + \
#                                    str(hptrt.parameter_dictionary['r-'][0][1]) + ')$',color='r',fontsize=15)
plt.savefig(path_to_helmpy + '/plots/treat_comp.png',format='png',dpi=500)
plt.show()

In [ ]:
example_lasttreat_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_1.txt')
example_final_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_final_prevalences_cluster_1.txt')

example_lasttreat_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_2.txt')
example_final_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_final_prevalences_cluster_2.txt')

p1,b1 = np.histogram(example_lasttreat_prev_data,bins='fd')
p2,b2 = np.histogram(example_final_prev_data,bins='fd')
p3,b3 = np.histogram(example_lasttreat_prev_data2,bins='fd')
p4,b4 = np.histogram(example_final_prev_data2,bins='fd')

plt.plot(0.5*(b1[:len(b1)-1]+b1[1:len(b1)]),p1/max(p1),color='r',alpha=0.4)
plt.plot(0.5*(b2[:len(b2)-1]+b2[1:len(b2)]),p2/max(p2),color='r')
plt.plot(0.5*(b3[:len(b3)-1]+b3[1:len(b3)]),p3/max(p3),color='b',alpha=0.4)
plt.plot(0.5*(b4[:len(b4)-1]+b4[1:len(b3)]),p4/max(p4),color='b')
plt.show()

In [ ]:
example_lasttreat_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'mig_trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_1.txt')
example_final_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'mig_trt_' + output_filename + \
                                                  '_final_prevalences_cluster_1.txt')

example_lasttreat_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'mig_trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_2.txt')
example_final_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'mig_trt_' + output_filename + \
                                                  '_final_prevalences_cluster_2.txt')

p1,b1 = np.histogram(example_lasttreat_prev_data,bins='fd')
p2,b2 = np.histogram(example_final_prev_data,bins='fd')
p3,b3 = np.histogram(example_lasttreat_prev_data2,bins='fd')
p4,b4 = np.histogram(example_final_prev_data2,bins='fd')

plt.plot(0.5*(b1[:len(b1)-1]+b1[1:len(b1)]),p1/max(p1),color='r',alpha=0.4)
plt.plot(0.5*(b2[:len(b2)-1]+b2[1:len(b2)]),p2/max(p2),color='r')
plt.plot(0.5*(b3[:len(b3)-1]+b3[1:len(b3)]),p3/max(p3),color='b',alpha=0.4)
plt.plot(0.5*(b4[:len(b4)-1]+b4[1:len(b4)]),p4/max(p4),color='b')
plt.show()

## 1.4 Adding individual non-compliance to the treatment rounds

The `helmpy` class also contains models for systematic individual non-compliance to the MDA programme in the form of the conditional probabilities of: individuals in the $j$-th grouping being treated in the $n$-th round given treatment in the $(n-1)$-th round $\alpha^j_{n,n-1}$ and being treated in the $n$-th round given non-treatment in the $(n-1)$-th round $\beta^j_{n,n-1}$ in a Markov model such that the probability of an individual in the $j$-th grouping is treated in the $n$-th round may be written as

$$p_{j,n} = \alpha^j_{n,n-1} p_{j,n-1} + \beta^j_{n,n-1} [1 - p_{j,n-1}]\,.$$

To demonstrate how this model impacts the MDA scenario we have already been working with, let us once again re-initialise `helmpy`...

In [ ]:
hpcom = helmpy(
              'STH',                              # Set the disease type - types available are: 'STH', 'SCH' and 'LF'
              path_to_helmpy,                     # Set the path to the working directory
              suppress_terminal_output=False      # Set this to 'True' to remove terminal messages
              ) 

hpcom.parameter_dictionary['mu'] = [0.014,0.014]   # Human death rate (per year)
hpcom.parameter_dictionary['mu1'] = [0.5,0.5]      # Adult worm death rate (per year)
hpcom.parameter_dictionary['mu2'] = [26.0,26.0]    # Reservoir (eggs and larvae) death rate (per year)
hpcom.parameter_dictionary['R0'] = [3.5,2.1]       # Basic reproduction number within grouping
hpcom.parameter_dictionary['k'] = [0.3,0.5]        # Inverse-clumping factor within grouping
hpcom.parameter_dictionary['gam'] = [0.08,0.08]    # Density dependent fecundity: z = exp(-gam)
hpcom.parameter_dictionary['Np'] = [300,350]       # Number of people within grouping   
hpcom.parameter_dictionary['spi'] = [1,2]          # Spatial index number of grouping

hpcom.initial_conditions['M'] = [2.9,2.1]          # Initial mean total worm burden within grouping
hpcom.initial_conditions['FOI'] = [1.25,1.1]       # Initial force of infection (per year) within grouping

Having initialised another new instance, let us define the same 3 rounds of MDA with the same effective 60% coverage at years 15, 16 and 17 but with compliance parameters which indicate systematic individual non-compliance...

In [ ]:
comp_alpha_betas = [[0.6,0.4,0.97,0.05,0.97,0.05],     # A list of lists matching the chosen groupings which gives 
                    [0.6,0.4,0.97,0.05,0.97,0.05]]     # the alpha and beta parameters in the order of 
                                                       # alpha_1, beta_1, alpha_2,... in each, where the 
                                                       # first round alpha is an initial coverage fraction
    
treatment_times = [15.0,16.0,17.0]                     # A list of treatment times for all clusters

hpcom.add_treatment_prog(
                        treatment_times,         
                        compliance_params=comp_alpha_betas,
                        drug_efficacy=1.0
                        )

Now it is time to run the code and plot the same outputs at the end of the runs as before for comparison...

In [ ]:
runtime = 100.0                               # Set the total time of the run in years
realisations = 250                            # Set the number of stochastic realisations for the model
do_nothing_timescale = 0.01                   # Set a timescale (in years) short enough such that an individual 
                                              # is expected to stay in the same state
    
output_filename = 'default_example'           # Set a filename for the data to be output in data/
            
output_timesteps = [1000,2000,3000,9000]      # Optional - output binned worm burdens over whole population 
                                              # and realisations after a specified number of steps in time

hpcom.run_full_stoch(         
                    runtime,          
                    realisations,  
                    do_nothing_timescale,
                    'com_' + output_filename,  
                    timesteps_snapshot=output_timesteps
                    )

In [ ]:
example_output_data_com = np.loadtxt(path_to_helmpy + '/data/' + 'com_' + output_filename + '.txt')
example_output_data_trt = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + '.txt')

# Mean of ensemble in cluster 1 with non-compliance
plt.plot(example_output_data_com.T[0],example_output_data_com.T[1],color='r',\
         label=r'$R_0 =' + str(hpcom.parameter_dictionary['R0'][0]) + r'\,\,' + \
               r'k =' + str(hpcom.parameter_dictionary['k'][0]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hpcom.parameter_dictionary['Np'][0]) + r'\,\,' + \
               r'M(t_0) =' + str(hpcom.initial_conditions['M'][0]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hpcom.initial_conditions['FOI'][0]) + r'$')

# Mean of ensemble in cluster 2 with non-compliance
plt.plot(example_output_data_com.T[0],example_output_data_com.T[2],color='b',\
         label=r'$R_0 =' + str(hpcom.parameter_dictionary['R0'][1]) + r'\,\,' + \
               r'k =' + str(hpcom.parameter_dictionary['k'][1]) + r'\,\,' + \
               r'N_{\mathrm{p}} =' + str(hpcom.parameter_dictionary['Np'][1]) + r'\,\,' + \
               r'M(t_0) =' + str(hpcom.initial_conditions['M'][1]) + r'\,\,' + \
               r'\Lambda (t_0) =' + str(hpcom.initial_conditions['FOI'][1]) + r'$')

# 68% CLs of ensemble in cluster 1 with non-compliance
plt.plot(example_output_data_com.T[0],example_output_data_com.T[5],color='r')
plt.plot(example_output_data_com.T[0],example_output_data_com.T[7],color='r')

# 68% CLs of ensemble in cluster 2 with non-compliance
plt.plot(example_output_data_com.T[0],example_output_data_com.T[6],color='b')
plt.plot(example_output_data_com.T[0],example_output_data_com.T[8],color='b')

# Mean of ensemble in cluster 1 
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[1],color='r',alpha=0.4)

# Mean of ensemble in cluster 2
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[2],color='b',alpha=0.4)

# 68% CLs of ensemble in cluster 1
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[5],color='r',alpha=0.4)
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[7],color='r',alpha=0.4)

# 68% CLs of ensemble in cluster 2
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[6],color='b',alpha=0.4)
plt.plot(example_output_data_trt.T[0],example_output_data_trt.T[8],color='b',alpha=0.4)

axes = plt.gca()
axes.set_ylim([0.0,8.5])
axes.set_ylabel(r'$m(t)$')
axes.set_xlabel(r'$t-t_0$')

plt.legend(fontsize = 12, loc = 9)
#plt.text(50.0,1.0,r'$(r_+,r_-)=(' + str(hptrt.parameter_dictionary['r+'][0][1]) + ',' + \
#                                    str(hptrt.parameter_dictionary['r-'][0][1]) + ')$',color='r',fontsize=15)
plt.savefig(path_to_helmpy + '/plots/noncomptreat_comp.png',format='png',dpi=500)
plt.show()

In [ ]:
example_lasttreat_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_1.txt')
example_final_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_final_prevalences_cluster_1.txt')

example_lasttreat_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_2.txt')
example_final_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'trt_' + output_filename + \
                                                  '_final_prevalences_cluster_2.txt')

p1,b1 = np.histogram(example_lasttreat_prev_data,bins='fd')
p2,b2 = np.histogram(example_final_prev_data,bins='fd')
p3,b3 = np.histogram(example_lasttreat_prev_data2,bins='fd')
p4,b4 = np.histogram(example_final_prev_data2,bins='fd')

plt.plot(0.5*(b1[:len(b1)-1]+b1[1:len(b1)]),p1/max(p1),color='r',alpha=0.4)
plt.plot(0.5*(b2[:len(b2)-1]+b2[1:len(b2)]),p2/max(p2),color='r')
plt.plot(0.5*(b3[:len(b3)-1]+b3[1:len(b3)]),p3/max(p3),color='b',alpha=0.4)
plt.plot(0.5*(b4[:len(b4)-1]+b4[1:len(b3)]),p4/max(p4),color='b')
plt.show()

In [ ]:
example_lasttreat_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'com_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_1.txt')
example_final_prev_data = np.loadtxt(path_to_helmpy + '/data/' + 'com_' + output_filename + \
                                                  '_final_prevalences_cluster_1.txt')

example_lasttreat_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'com_' + output_filename + \
                                                  '_lasttreat_prevalences_cluster_2.txt')
example_final_prev_data2 = np.loadtxt(path_to_helmpy + '/data/' + 'com_' + output_filename + \
                                                  '_final_prevalences_cluster_2.txt')

p1,b1 = np.histogram(example_lasttreat_prev_data,bins='fd')
p2,b2 = np.histogram(example_final_prev_data,bins='fd')
p3,b3 = np.histogram(example_lasttreat_prev_data2,bins='fd')
p4,b4 = np.histogram(example_final_prev_data2,bins='fd')

plt.plot(0.5*(b1[:len(b1)-1]+b1[1:len(b1)]),p1/max(p1),color='r',alpha=0.4)
plt.plot(0.5*(b2[:len(b2)-1]+b2[1:len(b2)]),p2/max(p2),color='r')
plt.plot(0.5*(b3[:len(b3)-1]+b3[1:len(b3)]),p3/max(p3),color='b',alpha=0.4)
plt.plot(0.5*(b4[:len(b4)-1]+b4[1:len(b3)]),p4/max(p4),color='b')
plt.show()

## 1.5 Running with ageing process between bins

The default setting in `helmpy` is to not have people's worm counts 'ageing' out of bins. To include this ageing as a Poisson process which occurs at a rate controlled by the birth rate of new indviduals (with zero worms) into the cluster while keeping the number of people within each grouping constant for all time, we may adapt `helmpy` in the following way...

In [ ]:
hpage = helmpy(
            'STH',                              # Set the disease type - types available are: 'STH', 'SCH' and 'LF'
            path_to_helmpy,                     # Set the path to the working directory
            suppress_terminal_output=False      # Set this to 'True' to remove terminal messages
            )  

hpage.parameter_dictionary['mu'] = [0.014,0.014]      # Human death rate (per year)
hpage.parameter_dictionary['mu1'] = [0.5,0.5]         # Adult worm death rate (per year)
hpage.parameter_dictionary['mu2'] = [26.0,26.0]       # Reservoir (eggs and larvae) death rate 
hpage.parameter_dictionary['R0'] = [2.1,2.1]          # Basic reproduction number within grouping
hpage.parameter_dictionary['k'] = [0.3,0.3]           # Inverse-clumping factor within grouping
hpage.parameter_dictionary['gam'] = [0.08,0.08]       # Density dependent fecundity: z = exp(-gam)
hpage.parameter_dictionary['Np'] = [500,500]          # Number of people within grouping   
hpage.parameter_dictionary['spi'] = [1,1]             # Spatial index number of grouping

hpage.initial_conditions['M'] = [2.1,2.1]             # Initial mean total worm burden 
hpage.initial_conditions['FOI'] = [1.1,1.1]           # Initial force of infection (per year) 

...the key modification here is to include three new parameters which are...

In [ ]:
hp.parameter_dictionary['ari'] = [0,1]             # Group age-ordering index - 0,1,2,3,... 
hp.parameter_dictionary['brat'] = [2.0,2.0]        # Birth rate per year into grouping 0
hp.parameter_dictionary['Na'] = [10]               # Number of people ageing per event

In [ ]:
runtime = 100.0                               # Set the total time of the run in years
realisations = 250                            # Set the number of stochastic realisations for the model
do_nothing_timescale = 0.01                   # Set a timescale (in years) short enough such that an individual 
                                              # is expected to stay in the same state
    
output_filename = 'default_example'           # Set a filename for the data to be output in data/
            
output_timesteps = [1000,2000,3000,9000]      # Optional - output binned worm burdens over whole population 
                                              # and realisations after a specified number of steps in time

hpcom.run_full_stoch(         
                    runtime,          
                    realisations,  
                    do_nothing_timescale,
                    'age_' + output_filename,  
                    timesteps_snapshot=output_timesteps
                    )

Including the ageing process typically increases the runtime by around 40%.

## 1.6 Running large numbers of people and clusters

In order to run `helmpy` with a large number of people and clusters, it is advised to reduce the number of realisations to 1 as the internal vectorisation which optimises the code cannot handle matrices that are too large. The example below tests the runtime of helmpy with 500 people each in 40 clusters including migration and treatment rounds and emphasises the simplicity of using the `helmpy` class to instanciate a run...

In [ ]:
runtime = 100.0                              
realisations = 1                              
do_nothing_timescale = 0.01                                                 
    
output_filename = 'tests'                  

treatment_coverages = [[0.6,0.6,0.6] for j in range(0,40)]                          
treatment_times = [15.0,16.0,17.0]     

hptest = helmpy('STH',path_to_helmpy,suppress_terminal_output=True) 
hptest.parameter_dictionary['Np'] = [500 for j in range(0,40)]         
hptest.parameter_dictionary['spi'] = [j for j in range(0,40)]         
hptest.parameter_dictionary['r+'] = [[10.0*(i!=j) for i in range(0,40)] for j in range(0,40)] 
hptest.parameter_dictionary['r-'] = [[10.0*(i!=j) for i in range(0,40)] for j in range(0,40)]
hptest.add_treatment_prog(treatment_times,treatment_coverages=treatment_coverages,drug_efficacy=1.0) 

start_time = time.time()
hptest.run_full_stoch(runtime,realisations,do_nothing_timescale,output_filename)
end_time = time.time()

print('Runtime: ' + str(end_time-start_time))

...which appears to be around the 22 minute mark.

## 1.7 Fitting the simulation to Kato-Katz data for parameter inference and forecasting 

Using `helmpy`, a brute force method of fitting to data with simulations can be considered. If multiple candidate realisations of the simulation are acquired and then either accepted or rejected as posterior samples over initial conditions and transmission parameters then these can either be used for parameter inference or to run in a subsequent forecasting simulation according to their likelihood.

The Kato-Katz egg count data have a mean ${\rm E}({\rm egg})$ and variance ${\rm Var}({\rm egg})$, as independent summary statistics, respectively. An inference of the posterior distribution $p({\bf s}\vert {\cal D})$ over the summary statistics ${\bf s} =\{ {\rm E}({\rm egg}),{\rm Var}({\rm egg})\}$ given the data ${\cal D}$ can be used to obtain the full posterior $p({\bf x}\vert {\cal D})$ over the transmission parameters and initial conditions ${\bf x}$ given the data ${\cal D}$ in the following way

$p({\bf x}\vert {\cal D})=\int p({\bf x},{\bf s} \vert {\cal D}) \, {\rm d}{\bf s}$

$\qquad \quad =\int p({\bf x} \vert {\bf s}) p({\bf s} \vert {\cal D}) \, {\rm d}{\bf s}$

$\qquad \quad =\int p({\bf s}\vert {\bf x}) p({\bf x}) \frac{p({\bf s} \vert {\cal D})}{p({\bf s})} {\rm d}{\bf s}\,.$

Having marginalised over the inferred variance ${\rm Var}({\rm egg})$, which is assumed to be due to Kato-Katz diagnostic error, the subset of egg count means $\tilde{{\bf s}} =\{ {\rm E}({\rm egg})\}$ can simulated as a function of the ${\bf x}$, which we indicate by ${\bf f}({\bf x})$. For modelling reasons, the functional form of $p(\tilde{{\bf s}}\vert {\bf x})$ is assumed to be Gaussian

$$p({\bf s}\vert {\bf x}) = \frac{1}{\sqrt{\prod_i2\pi \epsilon^2}}\exp \left\{ -\sum_i\frac{[{\rm f}_i(x_i)-\tilde{s}_i]^2}{2\epsilon^2}\right\}\,,$$

with prior $p({\bf x})$ and $\frac{p(\tilde{{\bf s}} \vert {\cal D})}{p(\tilde{{\bf s}})}$ is proportional to the posterior over the summary parameters given the data. Furthermore, the simulator likelihood has a tolerance scale parameter $\epsilon$ which is unknown, therefore we need to input a range of its possible values (which we should do below) and select the one for which there is the greatest evidence.

The likelihood is automatically by `helmpy` once the summary statistics have been inferred and a subsequent simulation with parameter/initial condition samples has been run. To begin with our example, then we must first create a batch of mock Kato-Katz egg counts...

In [ ]:
meanegg = 30.0
varegg = 8000.0
kksamps = np.random.negative_binomial(meanegg**2.0/np.abs(varegg-meanegg),meanegg/varegg,size=1000)

Having done this, `helmpy` must then be initialised and given the necessary information for the fit, including the Kato-Katz $\lambda_{\rm epg}$ parameter and a list of tolerance-to-fitting parameters $\epsilon$ (discussed above)...

In [ ]:
hpfit = helmpy('STH',path_to_helmpy,suppress_terminal_output=False)  

hpfit.parameter_dictionary['Np'] = [1000]             # Number of people within grouping   
hpfit.parameter_dictionary['spi'] = [1]               # Spatial index number of grouping

hpfit.data_specific_parameters['KatoKatz'] = [3.0]                             # Kato-Katz egg count lambda_epg 
hpfit.data_specific_parameters['tolerances'] = [10.0**(-10.0+(float(i)*0.5)) \ # Range of possible tolerances input
                                                for i in range(0,20)]   

Having specified to this `helmpy` instance that it is reading Kato-Katz data through setting a value for `data_specific_parameters['KatoKatz']`, all we now need to to is run `fit_data`...

In [ ]:
data_from_file = [kksamps]                 # Input the data in a list structure equivalent to the input parameters

output_filename = 'default_example'        # Set a filename for the data to be output

walker_initconds = [[30.0,1.0],[9.0,0.1]]  # Parameter initial conditions [centre,width] for the ensemble MC walkers

plot_labels = ['Egg Mean','ln-Variance']   # Option to list a set of variable names (strings) in the same order 

hpfit.fit_data(data_from_file,           
               walker_initconds,         
               output_filename,          
               output_corner_plot=True,  
               plot_labels=plot_labels,  
               num_walkers=100,          
               num_iterations=500)       

The output for these inferred summary parameters has been stored in a text file but the current `helmpy` instance has also automatically stored these samples for comparison to simulation runs using the marginalised likelihood method described at the start of this subsection. To generate samples of the parameters/initial conditions and their corresponding likelihoods with respect to the data, and hence obtain posterior samples, all we need to is run this same instance of `helmpy` (using the parameter samples feature) and an output with likelihoods for each realisation will be generated. First we set the other parameters and include ageing...

In [ ]:
hpfit.parameter_dictionary['mu'] = [0.014]         # Human death rate (per year)
hpfit.parameter_dictionary['mu1'] = [0.5]          # Adult worm death rate (per year)
hpfit.parameter_dictionary['mu2'] = [26.0]         # Reservoir (eggs and larvae) death rate
hpfit.parameter_dictionary['ari'] = [0]            # Group age-ordering index - 0,1,2,3,... 
hpfit.parameter_dictionary['brat'] = [8.4]         # Birth rate per year into grouping 0
hpfit.parameter_dictionary['Na'] = [10]            # Number of people ageing per event

We can now also make use of the `posterior_samples` feature which allows for a set of samples of parameters and initial conditions to be input...

In [ ]:
realisations = 250                            

gams = np.random.uniform(0.001,0.01,size=realisations)
ks = 10.0**(np.random.uniform(-2.5,0.0,size=realisations))

hpfit.posterior_samples['ksamps'] = [ks]                                             # Initialisation with k samples
hpfit.posterior_samples['R0samps'] = [np.random.uniform(1.0,5.0,size=realisations)]  # Initialisation with R0 samples
hpfit.posterior_samples['gamsamps'] = [gams]                                         # Initialisation with gam samples
hpfit.posterior_samples['Msamps'] = [np.random.uniform(0.0,100.0,size=realisations)] # Initialisation with M samples
hpfit.posterior_samples['FOIsamps'] = [hpfit.posterior_samples['Msamps'][0]*\        # Initialisation with FOI samples
                                       hpfit.parameter_dictionary['mu1'][0]] 

Lastly, these samples may be run and their likelihoods with respect to the data will be calculated and output...

In [ ]:
runtime = 20.0 
do_nothing_timescale = 0.01 

hpfit.run_full_stoch(runtime,realisations,do_nothing_timescale,'fit_' + output_filename)

Let us now read in the output and visualise the parameter samples and their likelihoods with respect to the data in some scatter plots...

In [ ]:
fits = np.loadtxt(path_to_helmpy + '/data/' + 'fit_' + output_filename + '_likelihood_cluster_1.txt')

maximum_index = np.argmax(spec.logsumexp(fits,axis=0))
print('Maximum with index: ' + str(maximum_index))
print('Corresponding log tolerance value: ' + str(np.log10(hp.data_specific_parameters['tolerances'])[maximum_index]))
best_fits = fits[:,maximum_index]

plt.plot(np.log10(hpfit.data_specific_parameters['tolerances']),spec.logsumexp(fits,axis=0))
plt.show()

In [ ]:
for j in range(0,realisations):
    plt.scatter(hpfit.posterior_samples['Msamps'][0][j],\
                np.log10(hpfit.posterior_samples['ksamps'][0][j]),\
                color='Red',alpha=np.exp(best_fits[j]-np.max(best_fits)))
plt.show()

In [ ]:
for j in range(0,realisations):
    plt.scatter(hpfit.posterior_samples['R0samps'][0][j],\
                np.log10(hpfit.posterior_samples['ksamps'][0][j]),\
                color='Red',alpha=np.exp(best_fits[j]-np.max(best_fits)))
plt.show()

In [ ]:
for j in range(0,realisations):
    plt.scatter(hpfit.posterior_samples['gamsamps'][0][j],\
                np.log10(hpfit.posterior_samples['ksamps'][0][j]),\
                color='Red',alpha=np.exp(best_fits[j]-np.max(best_fits)))
plt.show()